Source code of every test for the task A with a BERT model

In [1]:
import numpy as np
import pandas as pd

# Load CSV files.
#CSV task A 
def getData():
    df_train_data = pd.read_csv("data/Training_Data/subtaskA_data_all.csv")
    df_train_answers = pd.read_csv("data/Training_Data/subtaskA_answers_all.csv")

    df_train = pd.merge(df_train_data,df_train_answers,on='id', how='left').drop(['id'], axis=1)
    
    df_dev_data = pd.read_csv("data/Dev_Data/subtaskA_dev_data.csv")
    df_dev_answers = pd.read_csv("data/Dev_Data/subtaskA_gold_answers.csv")

    df_dev = pd.merge(df_dev_data,df_dev_answers,on='id', how='left').drop(['id'], axis=1)

    df_test_data = pd.read_csv("data/Test_Data/subtaskA_test_data.csv")
    df_test_answers = pd.read_csv("data/Test_Data/subtaskA_gold_answers.csv")

    df_test= pd.merge(df_test_data,df_test_answers,on='id', how='left').drop(['id'], axis=1)
    
    return df_train, df_dev, df_test

df_train_A, df_dev_A, df_test_A = getData()



In [2]:
import spacy
nlp = spacy.load("en_core_web_sm")

Methods to pre-process the dataframe

In [3]:
def lemmatizer(text):
    """
    Receives a string as an input and lemmatizes it.
    """
    str = ""
    doc = nlp(text)
    for token in doc:
        str+=" "+token.lemma_
    return str 


def stemmatizer(text) :
    """
    Receive a string in input and stem it.
    """
    str = ""
    doc = nlp(text)
    for token in doc :
        str += ""+stemmer.stem(token.text)
    return str

def removeStopWords(text):
    """
    Receives a string and remove stop words from it.
    """
    str = ""
    doc = nlp(text)
    for token in doc:
        if(not token.is_stop):
            str+=" "+token.text
    return str 

def ngrams(text, n):
    """
    Receives a text and generates n-grams.
    """
    sequence=[]
    str = ""
    doc = nlp(text)
    for token in doc :
        sequence.append(token.text)
    return list(tuple([sequence[i] for i in range(i, i+n)]) for i in range(len(sequence)-n+1))

In [4]:
from nltk.corpus import wordnet as wn

In [36]:
def synsets(text, function):
    """
    Receives a text and return a list of synsets
    """
    text = text.replace("."," .")
    sequence=text.split()
    L = []
    for seq in sequence:
        L.append(tuple(function(s) for s in wn.synsets(seq)))
    return L

def getLemmasNames(synset):
    return [str(lemma.name()) for lemma in synset.lemmas()]

def getHypernyms(synset):
    return [s.name().split(".")[0] for s in synset.hypernyms() ]


    
synsets("He drinks apple.", getLemmasNames)
synsets("He drinks apple.", getHypernyms)

[(['chemical_element', 'noble_gas'], ['letter']),
 (['helping'],
  ['intemperance'],
  ['food', 'liquid'],
  ['body_of_water'],
  ['consumption'],
  ['consume'],
  ['consume'],
  ['honor'],
  ['steep'],
  ['use']),
 (['edible_fruit', 'pome'], ['apple_tree']),
 ()]

In [6]:
def pre_process(df, function):
    newdf = df[['sent0', 'sent1']]
    newdf.loc[:,"sent0"] = df.sent0.apply(function)
    newdf.loc[:,"sent1"] = df.sent1.apply(function)
    return newdf

Process of data frame, create subsample of it

In [15]:
def subsampleData():
    # subsample data 
    train = df_train_A.sample(n=1000, random_state=42)
    X_train = train[['sent0', 'sent1']]
    y_train = train['answer']

    return X_train, y_train

# use the dev set for testing
X_test = df_dev_A[['sent0', 'sent1']]
y_test = df_dev_A['answer']

In [27]:
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
def test_performance(model, x_test, y_test):
    y_pred = model.predict(x_test)
    print(classification_report(y_pred=y_pred, y_true=y_test))
    return f1_score(y_pred=y_pred, y_true=y_test, average="binary")

Importation of the BERT model

In [16]:
from transformers import BertModel
from bert_sklearn import BertClassifier

In [17]:
model = BertClassifier(max_seq_length=64, train_batch_size=16)
#model.num_mlp_layers = 3
model.max_seq_length = 64
model.epochs = 3
#model.learning_rate = 4e-5
                             
model

Building sklearn text classifier...


BertClassifier(max_seq_length=64, train_batch_size=16)

In [18]:
X_train_sample, y_train = subsampleData()

Fit with different preprocess type                                                                

In [19]:
X_train_sample.head()

,sent0,sent1
6252,a duck walks on three legs,a duck walks on two legs
4684,Jack's mom praised him because he broke the plate,Jack's mom condemned him because he broke the ...
1731,People use electricity to buy things,People use money to buy things
4742,"The speaker is damaged, thus I can't hear anyt...","The display is damaged, thus I can't hear anyt..."
4521,Santa Claus is the legend of the East,Santa Claus is the legend of the West


In [ ]:
model_classic = model.fit(X_train_sample, y_train)

With lemmas names

In [22]:
from functools import partial

In [23]:
pipe_fn = partial(synsets, function=getLemmasNames)
X_train = pre_process(X_train_sample,pipe_fn)
X_test_lemma = pre_process(X_test,pipe_fn)

X_train.head()

,sent0,sent1
6252,"[([angstrom, angstrom_unit, A], [vitamin_A, an...","[([angstrom, angstrom_unit, A], [vitamin_A, an..."
4684,"[(), ([ma, mama, mamma, mom, momma, mommy, mam...","[(), ([ma, mama, mamma, mom, momma, mommy, mam..."
1731,"[([people], [citizenry, people], [people], [mu...","[([people], [citizenry, people], [people], [mu..."
4742,"[(), ([speaker, talker, utterer, verbalizer, v...","[(), ([display, show], [display, exhibit, show..."
4521,"[([Santa_Claus, Santa, Kriss_Kringle, Father_C...","[([Santa_Claus, Santa, Kriss_Kringle, Father_C..."


In [24]:
model_lemma = model.fit(X_train, y_train)


Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint
train data size: 900, validation data size: 100


Validating: 100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:42<00:00,  3.29s/it]

Epoch 1, Train loss: 0.7369, Val loss: 0.7339, Val accy: 45.00%



Validating: 100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:41<00:00,  3.22s/it]

Epoch 2, Train loss: 0.7011, Val loss: 0.7060, Val accy: 45.00%



Validating: 100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:35<00:00,  2.72s/it]

Epoch 3, Train loss: 0.6988, Val loss: 0.6928, Val accy: 55.00%


In [28]:
f1 = test_performance(model_lemma, X_test, y_test)
print(f"f1 = {f1:.3f}")
f1 = test_performance(model_lemma, X_test_lemma, y_test)
print(f"f1 = {f1:.3f}")

Predicting: 100%|████████████████████████████████████████████████████████████████████| 125/125 [03:17<00:00,  1.58s/it]

              precision    recall  f1-score   support

           0       0.47      0.04      0.07       518
           1       0.48      0.95      0.64       479

    accuracy                           0.48       997
   macro avg       0.47      0.50      0.35       997
weighted avg       0.47      0.48      0.34       997

f1 = 0.636



Predicting: 100%|████████████████████████████████████████████████████████████████████| 125/125 [03:04<00:00,  1.48s/it]

              precision    recall  f1-score   support

           0       0.50      0.00      0.00       518
           1       0.48      1.00      0.65       479

    accuracy                           0.48       997
   macro avg       0.49      0.50      0.33       997
weighted avg       0.49      0.48      0.31       997

f1 = 0.649


With hypernyms

In [37]:
pipe_fn = partial(synsets, function=getHypernyms)
X_train = pre_process(X_train_sample,pipe_fn)
X_test_hypernyms = pre_process(X_test,pipe_fn)

X_train.head()

model_hypernyms = model.fit(X_train, y_train)

f1 = test_performance(model_hypernyms, X_test, y_test)
print(f"f1 = {f1:.3f}")
f1 = test_performance(model_hypernyms, X_test_hypernyms, y_test)
print(f"f1 = {f1:.3f}")

Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint
train data size: 900, validation data size: 100


Validating: 100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:37<00:00,  2.88s/it]

Epoch 1, Train loss: 0.7352, Val loss: 0.7272, Val accy: 45.00%



Validating: 100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:45<00:00,  3.49s/it]

Epoch 2, Train loss: 0.6997, Val loss: 0.7100, Val accy: 45.00%


...


Predicting: 100%|████████████████████████████████████████████████████████████████████| 125/125 [03:03<00:00,  1.47s/it]

              precision    recall  f1-score   support

           0       0.54      0.01      0.03       518
           1       0.48      0.99      0.65       479

    accuracy                           0.48       997
   macro avg       0.51      0.50      0.34       997
weighted avg       0.51      0.48      0.32       997

f1 = 0.647



Predicting: 100%|████████████████████████████████████████████████████████████████████| 125/125 [03:03<00:00,  1.46s/it]

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       518
           1       0.48      1.00      0.65       479

    accuracy                           0.48       997
   macro avg       0.24      0.50      0.32       997
weighted avg       0.23      0.48      0.31       997

f1 = 0.649



C:\Users\ellyn_vdxio7o\miniconda3\envs\tarProject\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ellyn_vdxio7o\miniconda3\envs\tarProject\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ellyn_vdxio7o\miniconda3\envs\tarProject\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg

To save a model

In [ ]:
#save model to disk
savefile = 'BERT_TaskA.bin'
model.save(savefile)